# Compressed Sparse Graph Routines (`scipy.sparse.csgraph`)

In [1]:
word_list = open('/usr/share/dict/words').readlines()
word_list = map(str.strip, word_list)

In [2]:
word_list = [word for word in word_list if len(word) == 3]
word_list = [word for word in word_list if word[0].islower()]
word_list = [word for word in word_list if word.isalpha()]
word_list = list(map(str.lower, word_list))
len(word_list)

1135

In [4]:
import numpy as np
word_list = np.asarray(word_list)
word_list.dtype   # these are unicode characters in Python 3

dtype('<U3')

In [6]:
word_list.sort()  # sort for quick searching later
word_bytes = np.ndarray((word_list.size, word_list.itemsize),
                        dtype='uint8',
                        buffer=word_list.data)
# each unicode character is four bytes long. We only need first byte
# we know that there are three characters in each word
word_bytes = word_bytes[:, ::word_list.itemsize//3]
word_bytes.shape

(1135, 3)

In [8]:
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix
hamming_dist = pdist(word_bytes, metric='hamming')
# there are three characters in each word
graph = csr_matrix(squareform(hamming_dist < 1.5 / 3))
i1 = word_list.searchsorted('ape')
i2 = word_list.searchsorted('man')
print(word_list[i1])
print(word_list[i2])

ape
man


In [9]:
from scipy.sparse.csgraph import dijkstra
distances, predecessors = dijkstra(graph, indices=i1,
                                   return_predecessors=True)
print(distances[i2])

5.0


In [10]:
path = []
i = i2
while i != i1:
    path.append(word_list[i])
    i = predecessors[i]
path.append(word_list[i1])
print(path[::-1])

['ape', 'ope', 'opt', 'oat', 'mat', 'man']


In [11]:
from scipy.sparse.csgraph import connected_components
N_components, component_list = connected_components(graph)
print(N_components)

4


In [12]:
[np.sum(component_list == i) for i in range(N_components)]

[1132, 1, 1, 1]

In [13]:
[list(word_list[np.nonzero(component_list == i)]) for i in range(1, N_components)]

[['edh'], ['its'], ['nth']]

In [14]:
distances, predecessors = dijkstra(graph, return_predecessors=True)
max_distance = np.max(distances[~np.isinf(distances)])
print(max_distance)

8.0


In [15]:
i1, i2 = np.nonzero(distances == max_distance)
list(zip(word_list[i1], word_list[i2]))

[('cwm', 'upo'),
 ('gnu', 'icy'),
 ('gnu', 'ump'),
 ('gnu', 'upo'),
 ('hei', 'upo'),
 ('hen', 'upo'),
 ('hep', 'upo'),
 ('hew', 'upo'),
 ('hex', 'upo'),
 ('hub', 'upo'),
 ('hug', 'upo'),
 ('hup', 'upo'),
 ('hyp', 'upo'),
 ('icy', 'gnu'),
 ('imp', 'mux'),
 ('imp', 'quo'),
 ('imp', 'yus'),
 ('imu', 'yus'),
 ('jet', 'upo'),
 ('jug', 'upo'),
 ('jut', 'upo'),
 ('keb', 'upo'),
 ('kef', 'upo'),
 ('keg', 'upo'),
 ('ken', 'upo'),
 ('kep', 'upo'),
 ('kex', 'upo'),
 ('mux', 'imp'),
 ('nun', 'upo'),
 ('our', 'upo'),
 ('peg', 'upo'),
 ('pen', 'upo'),
 ('pep', 'upo'),
 ('pew', 'upo'),
 ('pub', 'upo'),
 ('pug', 'upo'),
 ('pun', 'upo'),
 ('pup', 'upo'),
 ('pus', 'upo'),
 ('pya', 'upo'),
 ('pyx', 'upo'),
 ('quo', 'imp'),
 ('quo', 'upo'),
 ('ssu', 'upo'),
 ('uji', 'upo'),
 ('ump', 'gnu'),
 ('upo', 'cwm'),
 ('upo', 'gnu'),
 ('upo', 'hei'),
 ('upo', 'hen'),
 ('upo', 'hep'),
 ('upo', 'hew'),
 ('upo', 'hex'),
 ('upo', 'hub'),
 ('upo', 'hug'),
 ('upo', 'hup'),
 ('upo', 'hyp'),
 ('upo', 'jet'),
 ('upo', 'jug'

In [16]:
path = []
i = i2[0]
while i != i1[0]:
    path.append(word_list[i])
    i = predecessors[i1[0], i]
path.append(word_list[i1[0]])
print(path[::-1])

['cwm', 'cam', 'cay', 'cry', 'ary', 'ady', 'ado', 'udo', 'upo']
